# 1. Importing Libraries

In [1]:
import os

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

# 2.Reading the Data

In [2]:
PROJECT_DIR = r'C:\Users\Abhishek\Documents\flight-sagemaker-project'
DATA_DIR = 'data'


In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,file_name)
    return pd.read_csv(file_path)

In [4]:
flights = get_data('flight_price')
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [12]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The Dataset contains 10,683 rows and 11 features
- Columns 'Route' and 'Total_stops' have missing value each
- the data type of some features isn't appropriate

# 3.Preliminary Analysis

### 3.1 Check Data Types

In [14]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [16]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [14]:
value = flights.Date_of_Journey.iloc[6]
value

'12/03/2019'

In [16]:
value = flights.Dep_Time.iloc[6]
value

'18:55'

In [18]:
value = flights.Arrival_Time.iloc[6]
value

'10:25 13 Mar'

### 3.2 Check for Duplicates

In [21]:
flights.duplicated().sum()

220

In [18]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(["Airline","Date_of_Journey","Source","Destination"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observation

- The type of 'Date_of_journey','Dep_Time' and 'Arrival_Time' should be changed to datetime
- The type of 'Duration' and 'Total_stops' is mixed. It should be numeric type
- There are 220 Duplicates, These should be removed

# 4.Detailed Analysis 

### Airline

In [35]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [22]:
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

### Date_of_Journey

In [28]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

- Date_of_Journey column have inappropriate datatype

In [37]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source

In [79]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [84]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Route

In [91]:
flights.Route.unique()

array(['BLR → DEL', 'CCU → IXR → BBI → BLR', 'DEL → LKO → BOM → COK',
       'CCU → NAG → BLR', 'BLR → NAG → DEL', 'CCU → BLR',
       'BLR → BOM → DEL', 'DEL → BOM → COK', 'DEL → BLR → COK',
       'MAA → CCU', 'CCU → BOM → BLR', 'DEL → AMD → BOM → COK',
       'DEL → PNQ → COK', 'DEL → CCU → BOM → COK', 'BLR → COK → DEL',
       'DEL → IDR → BOM → COK', 'DEL → LKO → COK',
       'CCU → GAU → DEL → BLR', 'DEL → NAG → BOM → COK',
       'CCU → MAA → BLR', 'DEL → HYD → COK', 'CCU → HYD → BLR',
       'DEL → COK', 'CCU → DEL → BLR', 'BLR → BOM → AMD → DEL',
       'BOM → DEL → HYD', 'DEL → MAA → COK', 'BOM → HYD',
       'DEL → BHO → BOM → COK', 'DEL → JAI → BOM → COK',
       'DEL → ATQ → BOM → COK', 'DEL → JDH → BOM → COK',
       'CCU → BBI → BOM → BLR', 'BLR → MAA → DEL',
       'DEL → GOI → BOM → COK', 'DEL → BDQ → BOM → COK',
       'CCU → JAI → BOM → BLR', 'CCU → BBI → BLR', 'BLR → HYD → DEL',
       'DEL → TRV → COK', 'CCU → IXR → DEL → BLR',
       'DEL → IXU → BOM → COK', 'CCU 

- We have route data in Source and destination and total_no_of_stops

### Dep_time

In [101]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [30]:
pd.to_datetime(flights.Dep_Time,format='%H:%M').dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival_Time

In [56]:
flights.Arrival_Time.unique()

array(['01:10 22 Mar', '13:15', '04:25 10 Jun', ..., '06:50 10 Mar',
       '00:05 19 Mar', '21:20 13 Mar'], dtype=object)

- there are some values that contains date in it we dont need the date 

In [67]:
pd.to_datetime(flights.Arrival_Time.str.slice(0,5),format='%H:%M').dt.time

0        01:10:00
1        13:15:00
2        04:25:00
3        23:30:00
4        21:35:00
           ...   
10678    22:25:00
10679    23:20:00
10680    11:20:00
10681    14:10:00
10682    19:15:00
Name: Arrival_Time, Length: 10683, dtype: object

### Duration

In [82]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [94]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    
)

6474    5m
Name: Duration, dtype: object

In [106]:
flights.iloc[6474]

Airline                        Air India
Date_of_Journey                6/03/2019
Source                            Mumbai
Destination                    Hyderabad
Route              BOM → GOI → PNQ → HYD
Dep_Time                           16:50
Arrival_Time                       16:55
Duration                              5m
Total_Stops                      2 stops
Additional_Info                  No info
Price                              17327
Name: 6474, dtype: object

- The observation indexed 6474 has duration of 5 minutes. This is clearly Wrong Entry. Will delete this observation

In [117]:
(
    flights
    .Duration
    .drop(index=[6474])
    .str.split(" ",expand=True)
    .set_axis(["hour", "minute"],axis=1)
    .assign(
        hour=lambda df_ : (
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

### Total_Stops

In [170]:
flights.Total_Stops

0        non-stop
1         2 stops
2         2 stops
3          1 stop
4          1 stop
           ...   
10678    non-stop
10679    non-stop
10680    non-stop
10681    non-stop
10682     2 stops
Name: Total_Stops, Length: 10683, dtype: object

In [182]:
flights.loc[flights.Total_Stops.isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [192]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
    .unique()
)

array([ 0.,  2.,  1.,  3., nan,  4.])

# 5 Cleaning Operation

In [51]:
# FUNCTION TO CONVERT DURATION FORMAT FROM ("%h %m") TO (%m)
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ",expand=True)
        .set_axis(["hour", "minute"],axis=1)
        .assign(
            hour=lambda df_ : (
                df_
                .hour
                .str.replace("h","")
                .astype(int)
                .mul(60)
            ),
            minute = lambda df_: (
                df_
                .minute
                .str.replace("m","")
                .fillna("0")
                .astype(int)
            )
        )
        .sum(axis=1)
    )

In [53]:
def clean_data(df):
    df_cleaned = (
        df
        .drop(index=[6474])
        .drop_duplicates()
        .rename(columns=str.lower)
    )

    # Apply string stripping to object columns
    df_cleaned = df_cleaned.assign(**{
        col: df_cleaned[col].str.strip()
        for col in df_cleaned.select_dtypes(include='O').columns
    })

    # Apply other transformations
    df_cleaned = df_cleaned.assign(
        airline=lambda df_: (
            df_
            .airline
            .str.replace(" Premium economy", "")
            .str.replace(" Business", "")
            .str.title()
        ),
        date_of_journey=lambda df_: pd.to_datetime(df_.date_of_journey, dayfirst=True),
        dep_time=lambda df_: pd.to_datetime(df_.dep_time, format='%H:%M').dt.time,
        arrival_time=lambda df_: pd.to_datetime(df_.arrival_time.str.slice(0, 5), format='%H:%M').dt.time,
        duration=lambda df_: convert_to_minutes(df_.duration),
        total_stops=lambda df_: (
            df_
            .total_stops
            .replace("non-stop", "0")
            .str.replace(" stops?", "", regex=True)
            .pipe(pd.to_numeric)
        ),
        additional_info=lambda df_: df_.additional_info.replace("No info", "No Info")
    )

    # Drop unnecessary columns
    df_cleaned = df_cleaned.drop(columns="route")

    return df_cleaned


In [55]:
flights_cleaned = clean_data(flights)
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


# 6. Split The Data

In [57]:
flights_final = flights_cleaned.sample(1000)

In [65]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [67]:
X_, X_test, y_ , y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_,y_,test_size=0.2,random_state=42)

print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


# 7. Export The Subsets

In [74]:
def export_data(X,y,name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,file_name)
    X.join(y).to_csv(file_path, index=False)

    return pd.read_csv(file_path).head()


In [76]:
export_data(X_train,y_train,"train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-06-27,Delhi,Cochin,20:15:00,19:15:00,1380,2.0,No Info,13591
1,Jet Airways,2019-04-21,Banglore,Delhi,18:55:00,22:00:00,185,0.0,In-flight meal not included,4544
2,Multiple Carriers,2019-05-27,Delhi,Cochin,17:45:00,01:30:00,465,1.0,No Info,6795
3,Jet Airways,2019-05-06,Kolkata,Banglore,06:30:00,18:15:00,705,1.0,In-flight meal not included,10844
4,Goair,2019-06-09,Delhi,Cochin,10:35:00,19:35:00,540,1.0,No Info,5281


In [80]:
export_data(X_val,y_val,"val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-06-09,Kolkata,Banglore,16:50:00,13:45:00,1255,2.0,No Info,11752
1,Jet Airways,2019-06-01,Delhi,Cochin,14:35:00,12:35:00,1320,2.0,In-flight meal not included,10919
2,Vistara,2019-05-12,Kolkata,Banglore,20:20:00,23:25:00,1625,1.0,No Info,8085
3,Jet Airways,2019-05-24,Kolkata,Banglore,06:30:00,04:40:00,1330,1.0,In-flight meal not included,8586
4,Multiple Carriers,2019-06-27,Delhi,Cochin,13:15:00,22:30:00,555,1.0,No Info,8018


In [82]:
export_data(X_test,y_test,"test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-01,Banglore,Delhi,08:20:00,11:20:00,180,0.0,In-flight meal not included,7229
1,Indigo,2019-05-09,Delhi,Cochin,07:15:00,12:30:00,315,1.0,No Info,5636
2,Indigo,2019-06-09,Kolkata,Banglore,09:20:00,14:15:00,295,1.0,No Info,11465
3,Jet Airways,2019-04-01,Kolkata,Banglore,20:25:00,22:05:00,1540,1.0,No Info,12121
4,Air India,2019-06-09,Delhi,Cochin,06:05:00,09:25:00,1640,1.0,No Info,7690
